In [1]:
from tensorflow.keras.layers import Input, Softmax, Dense, Dropout
from tensorflow.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import re

from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

from keras import regularizers

import nltk
from nltk import word_tokenize

Using TensorFlow backend.


In [2]:
idfilename = 'training_data/id.txt'
datadirname = 'training_data/feat/'
labelfilename = 'training_label.json'

with open("DIC_word_index.json") as f:
    DIC_word_index = json.load(f)
    
with open("DIC_index_word.json") as f:
    DIC_index_word = json.load(f)

In [3]:
# loading training data
encode_x = []
video_id = {}
for i,video_name in enumerate(open(idfilename)):
    video_name = video_name[:-1]
    x = np.load(datadirname + video_name + ".npy")
    encode_x.append(x)
    video_id[video_name] = i

encode_x = np.array(encode_x)
    
TRAIN_SZ = len(encode_x)
decode_x = [[]]*TRAIN_SZ
decode_y = [[]]*TRAIN_SZ

In [4]:
def Sent2Seq(sent):
    #print(sent)    
    tokens = word_tokenize(sent.lower())
    ret = []
    for word in tokens:
        ret.append(DIC_word_index[word])
    return ret

BOS = "<bos>" # index is 1
EOS = "<eos>" # index is 2

VOCAB_SZ = len(DIC_word_index)
MAX_SEQ_LEN = 0;
# loading decoder data
rawlabels = json.load(open(labelfilename, 'r'))
for data in rawlabels:    
    index = video_id[data['id']]
    #print(index)
    sent =  data['caption'][0] # select one sentence for now
    # TODO: implement Sent2Seq
    decode_x[index] = [1] + Sent2Seq(sent)
    decode_y[index] = Sent2Seq(sent) + [2]
    if(len(decode_x[index]) > MAX_SEQ_LEN):
        MAX_SEQ_LEN = len(decode_x[index])


In [5]:
print(decode_x[0])
print(decode_y[0]) #one shift from decode_x

[1, 4, 5, 6, 7, 4, 8, 9]
[4, 5, 6, 7, 4, 8, 9, 2]


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

MAX_SEQ_LEN = 15

decode_x = pad_sequences(decode_x, maxlen=MAX_SEQ_LEN, padding='post', truncating='pre')
decode_y = pad_sequences(decode_y, maxlen=MAX_SEQ_LEN, padding='post', truncating='pre')
decode_y = to_categorical(decode_y, num_classes=VOCAB_SZ)
decode_x = to_categorical(decode_x, num_classes=VOCAB_SZ)


In [7]:
print(encode_x.shape)
print(decode_x.shape)
print(decode_y.shape)

(1450, 80, 4096)
(1450, 15, 6087)
(1450, 15, 6087)


In [8]:
latent_dim = 256
#=================encoder====================#
encoder_inputs = Input(shape=(80, 4096))
encoder = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
#=================decoder====================#
decoder_inputs = Input(shape=(None,VOCAB_SZ))
decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_ , _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(VOCAB_SZ, activation='softmax', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
decoder_outputs = decoder_dense(decoder_outputs)
#=============================================
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 80, 4096)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 6087)   0                                            
__________________________________________________________________________________________________
cu_dnnlstm (CuDNNLSTM)          [(None, 80, 256), (N 4458496     input_1[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        [(None, None, 256),  6497280     input_2[0][0]                    
                                                                 cu_dnnlstm[0][1]                 
          

In [10]:
optimizer = Adam(lr=0.001)

lrreduc = ReduceLROnPlateau(monitor='loss', factor=0.5,\
                             patience=5, min_lr=0.00001, verbose=1, cooldown=5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

model.fit([encode_x, decode_x], decode_y, validation_split=0.1, batch_size=1, epochs=50, callbacks=[lrreduc])

Train on 1305 samples, validate on 145 samples
Epoch 1/50
1305/1305 [==============================] - 17s 13ms/step - loss: 4.3409 - acc: 0.4282 - val_loss: 4.0182 - val_acc: 0.5025
Epoch 2/50
1305/1305 [==============================] - 15s 11ms/step - loss: 3.7068 - acc: 0.5252 - val_loss: 3.8304 - val_acc: 0.5172
Epoch 3/50
1305/1305 [==============================] - 15s 12ms/step - loss: 3.5495 - acc: 0.5386 - val_loss: 3.7451 - val_acc: 0.5223
Epoch 4/50
1305/1305 [==============================] - 15s 11ms/step - loss: 3.4535 - acc: 0.5449 - val_loss: 3.7597 - val_acc: 0.5191
Epoch 5/50
1305/1305 [==============================] - 15s 11ms/step - loss: 3.3851 - acc: 0.5505 - val_loss: 3.7321 - val_acc: 0.5315
Epoch 6/50
1305/1305 [==============================] - 15s 12ms/step - loss: 3.3410 - acc: 0.5545 - val_loss: 3.7148 - val_acc: 0.5324
Epoch 7/50
1305/1305 [==============================] - 15s 12ms/step - loss: 3.3035 - acc: 0.5562 - val_loss: 3.7488 - val_acc: 0.5241
E

In [11]:
#==============inference setup===================#
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [12]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty decode_y seq
    # y_seq shape : (1, 1, 6087)
    y_seq = np.zeros((1, 1, VOCAB_SZ))
    y_seq[0, 0, DIC_word_index["<bos>"]] = 1
    
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        #output_tokens shape : (1, 1, 6087)
        #output_tokens[0, -1, :] shape : (6087, )
        output_tokens, h, c = decoder_model.predict(  
            [y_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :] )
        sampled_token = DIC_index_word[str(sampled_token_index)]
        if(sampled_token!='<eos>'):
            decoded_sentence.append(sampled_token)
        
        #Exit Condition :either hit max length or find stop char
        
        if(sampled_token == '<eos>' or
          len(decoded_sentence) > MAX_SEQ_LEN):
            stop_condition = True
        
        #Update y_seq
        y_seq = np.zeros((1, 1, VOCAB_SZ))
        y_seq[0, 0, sampled_token_index] = 1
        
        #Update states
        states_value = [h, c]
        
    return decoded_sentence

In [13]:
idfilename = 'testing_data/id.txt'
datadirname = 'testing_data/feat/'
labelfilename = 'testing_label.json'

encode_x = []
video_id = []
for i,lb in enumerate(open(idfilename)):
    lb = lb[:-1]
    encode_x.append(np.load(datadirname + lb +".npy"))
    video_id.append(lb)
    
out_labels = []
for indexx in range(len(encode_x)):
    sent = decode_sequence(np.array([encode_x[indexx]]))
    sent = " ".join(sent)
    print(sent)
    out_labels.append(sent)

a woman is playing a .
a man is playing a .
a person is playing a .
a woman is playing a .
a woman is playing a .
a person is playing .
a person is playing .
a cat is playing .
a man is playing a .
a woman is cutting a .
someone is cutting a bowl of a bowl .
a woman is playing a .
a man is playing a .
a woman is playing a .
someone is cutting a bowl of a bowl .
a man is playing a .
a man is playing a .
a man is playing a .
a man is playing a a .
a woman is playing a .
a man is playing a .
a man is playing .
a man is playing a large .
a person is playing a .
a woman is cutting a bowl .
a person is playing a .
a woman is playing a .
a person is playing a .
a man is playing a .
a man is playing a .
a woman is cutting a bowl .
a woman is cutting a .
a man is playing a .
a man is playing a .
a woman is cutting a bowl .
a man is playing a .
a man is playing a .
a man is playing a large .
a boy is playing .
a woman is cutting a bowl .
two man is playing a .
a woman is cutting a .
a man is pla

In [14]:
with open('MODELTEST_testing.txt', 'w') as f:
    for i in range(len(encode_x)):
        f.write(video_id[i] + ',' + out_labels[i] + '\n')

In [15]:
#====================TESTING===================#

idfilename_t = 'testing_data/id.txt'
datadirname_t = 'testing_data/feat/'

# loading testing data
encode_x_t = []
video_id_t = {}
for i,video_name in enumerate(open(idfilename_t)):
    #lb contains '\n', therefore lb[:-1]
    video_name = video_name[:-1]
    x = np.load(datadirname_t + video_name + ".npy")
    encode_x_t.append(x)
    video_id_t[video_name] = i

In [16]:
#============predict the caption============#
OUTPUTS = []
for X in encode_x_t:
    X = np.array([X])
    Y = decode_sequence(X)
    OUTPUTS.append(Y)

In [21]:
#============to json============#
predict_label = []
with open('predict_label.txt', 'w') as f:
    for video_name, _id in video_id_t.items():
        tokens = OUTPUTS[int(_id)]
        predict = " ".join(tokens)
        
        if(predict[-1] == "."):
            predict  = predict[:-1]
        #print(OUTPUTS[int(_id)])
        #input("")
        f.write(str(video_name) + "," + predict +"\n")
        print(predict)
    

['a', 'woman', 'is', 'playing', 'a', '.']
a woman is playing a 
['a', 'man', 'is', 'playing', 'a', '.']
a man is playing a 
['a', 'person', 'is', 'playing', 'a', '.']
a person is playing a 
['a', 'woman', 'is', 'playing', 'a', '.']
a woman is playing a 
['a', 'woman', 'is', 'playing', 'a', '.']
a woman is playing a 
['a', 'person', 'is', 'playing', '.']
a person is playing 
['a', 'person', 'is', 'playing', '.']
a person is playing 
['a', 'cat', 'is', 'playing', '.']
a cat is playing 
['a', 'man', 'is', 'playing', 'a', '.']
a man is playing a 
['a', 'woman', 'is', 'cutting', 'a', '.']
a woman is cutting a 
['someone', 'is', 'cutting', 'a', 'bowl', 'of', 'a', 'bowl', '.']
someone is cutting a bowl of a bowl 
['a', 'woman', 'is', 'playing', 'a', '.']
a woman is playing a 
['a', 'man', 'is', 'playing', 'a', '.']
a man is playing a 
['a', 'woman', 'is', 'playing', 'a', '.']
a woman is playing a 
['someone', 'is', 'cutting', 'a', 'bowl', 'of', 'a', 'bowl', '.']
someone is cutting a bowl of a

In [18]:
#video_id_t

In [20]:
import os
os.system("python bleu_eval.py predict_label.txt")

0